In [173]:
import re
import os
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from pprint import pprint

# Split speech in sentences

### Speeches to sentences with labels

In [174]:
path1 = r"Talerne\Margrethe"
path2 = r"Talerne\Statsministeren"

def sentence_and_label(path):

    sentences = {}

    for file in os.listdir(path):
        filename = os.path.join(path, file)

        # Label the sentence
        if "Margrethe" in file:
            label = 0
        else:
            label = 1

        with open(filename, encoding='utf-8') as speech:
            speech = speech.read()
        
        speech = re.sub("[,*\"'-]"," ",speech)

        speech = speech.lower()

        tokens = nltk.sent_tokenize(speech, language="danish")

        for sentence in tokens:
            sentence = re.sub("[.:;?!`–”«»%…‘’/()]"," ",sentence) 
            sentence = sentence.strip() # Uden mellemrum forrest eller bagerst
            sentences[sentence] = label
        
    return sentences

Margrethe_sentences_with_labels = sentence_and_label(path1)
Statsministeren_sentences_with_labels = sentence_and_label(path2)

In [175]:
# from pprint import pprint

# # Pretty-print to look nice
# pprint(Margrethe_sentences_with_labels)
# pprint(Statsministeren_sentences_with_labels)


In [176]:
all_sentences = Margrethe_sentences_with_labels | Statsministeren_sentences_with_labels
# pprint(all_sentences)
print(len(all_sentences))

10731


In [177]:
# Inspect the tokens that look suspicious
for sentence in all_sentences:
    tokens = word_tokenize(sentence)
    # Look for weird tokens
    weird_tokens = [t for t in tokens if not t.isalnum()]
    if weird_tokens:
        print("Weird tokens found:", weird_tokens)

### Separate keys and values into two lists (For vectorization)

In [178]:
sentences_list = []
labels_list = []

for sentence in all_sentences:
    sentences_list.append(sentence)
    labels_list.append(all_sentences[sentence])

print(sentences_list)
print(labels_list)

['for min familie og for mig selv blev dette år skelsættende ved min fader kong frederiks sygdom og død', 'den dybe sorg  der ramte os  følte vi  at hele folket tog del i  og jeg kan ikke begynde denne min første nytårshilsen uden at bringe en tak for al den varme og sympati  som blev prins henrik og mig  men ikke mindst min moder  dronning ingrid  til del', 'de hjertelige følelser  som i de tunge dage støttede og løftede os  har jeg siden i årets løb mødt så mange gange  og det har været mig en glæde og inspiration i min gerning som danmarks dronning', '1972 vil blive et historisk år på mange måder', 'i danmarkshistorien vil året blive husket for den betydningsfulde beslutning  som det danske folk traf om danmarks medlemskab af det udvidede europæiske fællesskab', 'hermed træder vi ind i et nært samarbejde med de vigtigste vesteuropæiske nationer  vi går ind til det med store forhåbninger  men selvsagt også med en bevidsthed om  at dette vil stille krav til dansk kultur  og samfundsli

In [179]:
print(len(sentences_list))

10731


# UNIGRAM

In [180]:
unigram_dict = {}
unigram_set = set()

for sentence in sentences_list:
    sentence_tokens = [word for word in word_tokenize(sentence)]
    
    for word in sentence_tokens:
        if word in unigram_set:
            unigram_dict[word] += 1
        else:
            unigram_dict[word] = 1
            unigram_set.add(word)

len(unigram_dict)

11591

In [196]:
pprint(unigram_dict)

{'00': 1,
 '000': 36,
 '1': 11,
 '10': 30,
 '100': 25,
 '1000': 3,
 '11': 10,
 '12': 12,
 '1200': 1,
 '13': 1,
 '14': 6,
 '140': 1,
 '144': 1,
 '15': 11,
 '150': 4,
 '16': 1,
 '1600': 1,
 '17': 3,
 '1700': 1,
 '1788': 1,
 '18': 9,
 '1849': 1,
 '1864': 1,
 '1890': 1,
 '19': 1,
 '1900': 2,
 '1920': 6,
 '1930': 1,
 '1937': 1,
 '1943': 3,
 '1944': 1,
 '1945': 7,
 '1950': 2,
 '1962': 2,
 '1966': 1,
 '1968': 1,
 '1969': 1,
 '1970': 6,
 '1972': 9,
 '1973': 5,
 '1974': 8,
 '1975': 4,
 '1976': 4,
 '1977': 7,
 '1978': 8,
 '1979': 12,
 '1980': 5,
 '1981': 3,
 '1982': 6,
 '1983': 16,
 '1984': 11,
 '1985': 8,
 '1986': 7,
 '1987': 2,
 '1988': 5,
 '1989': 5,
 '1990': 12,
 '1991': 4,
 '1992': 7,
 '1993': 5,
 '1994': 3,
 '1995': 4,
 '1997': 3,
 '1998': 6,
 '1999': 6,
 '2': 11,
 '20': 15,
 '200': 9,
 '2001': 4,
 '2002': 3,
 '2003': 1,
 '2004': 5,
 '2005': 6,
 '2006': 4,
 '2007': 3,
 '2008': 8,
 '2009': 7,
 '2010': 4,
 '2011': 5,
 '2012': 4,
 '2013': 2,
 '2014': 7,
 '2015': 4,
 '2016': 3,
 '2017': 2,
 '2

In [207]:
unigram_dict_forkortet = {}

for unigram in unigram_dict.keys():
    if unigram_dict[unigram] > 1:
        unigram_dict_forkortet[unigram] = unigram_dict[unigram] 

unigram_dict_forkortet

{'for': 2920,
 'min': 246,
 'familie': 121,
 'og': 5600,
 'mig': 217,
 'selv': 407,
 'blev': 253,
 'dette': 108,
 'år': 963,
 'skelsættende': 8,
 'ved': 528,
 'fader': 4,
 'kong': 6,
 'frederiks': 3,
 'sygdom': 28,
 'død': 12,
 'den': 1560,
 'dybe': 12,
 'sorg': 15,
 'der': 2078,
 'ramte': 10,
 'os': 1268,
 'følte': 13,
 'vi': 4528,
 'at': 4158,
 'hele': 252,
 'folket': 5,
 'tog': 28,
 'del': 140,
 'i': 4606,
 'jeg': 1220,
 'kan': 1248,
 'ikke': 1663,
 'begynde': 23,
 'denne': 130,
 'første': 88,
 'nytårshilsen': 49,
 'uden': 168,
 'bringe': 60,
 'en': 2087,
 'tak': 190,
 'al': 43,
 'varme': 55,
 'sympati': 7,
 'som': 1712,
 'prins': 51,
 'henrik': 12,
 'men': 1155,
 'mindst': 87,
 'moder': 11,
 'dronning': 27,
 'ingrid': 9,
 'til': 2555,
 'de': 2208,
 'hjertelige': 8,
 'følelser': 9,
 'tunge': 15,
 'dage': 51,
 'støttede': 3,
 'løftede': 2,
 'har': 2038,
 'siden': 127,
 'årets': 56,
 'løb': 29,
 'mødt': 33,
 'så': 964,
 'mange': 642,
 'gange': 30,
 'det': 4769,
 'været': 208,
 'glæde'

In [208]:
len(unigram_dict_forkortet)

5590

# BIGRAM

In [182]:
bigram_dict = {}
bigram_set = set()

for sentence in sentences_list:
    sentence_tokens = [word for word in word_tokenize(sentence)]
    
    for i in range(len(sentence_tokens)-1):
        bigram = sentence_tokens[i] + ' ' + sentence_tokens[i+1]
        if bigram in bigram_set:
            bigram_dict[bigram] += 1
        else:
            bigram_dict[bigram] = 1
            bigram_set.add(bigram)

In [183]:
len(bigram_dict)

60904

In [209]:
# bigram_dict

In [210]:
bigram_dict_forkortet = {}

for bigram in bigram_dict.keys():
    if bigram_dict[bigram] > 1:
        bigram_dict_forkortet[bigram] = bigram_dict[bigram] 

bigram_dict_forkortet

{'for min': 13,
 'min familie': 49,
 'familie og': 31,
 'og for': 108,
 'for mig': 33,
 'mig selv': 8,
 'selv blev': 4,
 'dette år': 14,
 'ved min': 2,
 'min fader': 2,
 'sygdom og': 6,
 'og død': 2,
 'sorg der': 2,
 'følte vi': 5,
 'vi at': 35,
 'at hele': 3,
 'tog del': 2,
 'del i': 16,
 'i og': 11,
 'og jeg': 122,
 'jeg kan': 27,
 'kan ikke': 63,
 'ikke begynde': 2,
 'begynde denne': 2,
 'min første': 2,
 'første nytårshilsen': 2,
 'uden at': 49,
 'at bringe': 8,
 'bringe en': 6,
 'en tak': 49,
 'tak for': 101,
 'for al': 15,
 'al den': 16,
 'den varme': 13,
 'varme og': 18,
 'og sympati': 2,
 'sympati som': 2,
 'som blev': 10,
 'prins henrik': 11,
 'henrik og': 10,
 'og mig': 24,
 'men ikke': 14,
 'ikke mindst': 82,
 'min moder': 10,
 'moder dronning': 8,
 'dronning ingrid': 9,
 'til del': 9,
 'følelser som': 3,
 'som i': 78,
 'i de': 116,
 'de tunge': 4,
 'os har': 10,
 'har jeg': 29,
 'siden i': 2,
 'i årets': 14,
 'årets løb': 11,
 'mødt så': 2,
 'så mange': 69,
 'mange gange': 

In [211]:
len(bigram_dict_forkortet)

15268

# TRIGRAM

In [185]:
trigram_dict = {}
trigram_set = set()
for sentence in sentences_list:
    sentence_tokens = [word for word in word_tokenize(sentence)]
    
    for i in range(len(sentence_tokens)-2):
        trigram = sentence_tokens[i] + ' ' + sentence_tokens[i+1] + ' ' + sentence_tokens[i+2]
        if trigram in trigram_set:
            trigram_dict[trigram] += 1
        else:
            trigram_dict[trigram] = 1
            trigram_set.add(trigram)

In [205]:
trigram_dict

{'for min familie': 7,
 'min familie og': 16,
 'familie og for': 2,
 'og for mig': 2,
 'for mig selv': 1,
 'mig selv blev': 1,
 'selv blev dette': 1,
 'blev dette år': 1,
 'dette år skelsættende': 1,
 'år skelsættende ved': 1,
 'skelsættende ved min': 1,
 'ved min fader': 1,
 'min fader kong': 1,
 'fader kong frederiks': 1,
 'kong frederiks sygdom': 1,
 'frederiks sygdom og': 1,
 'sygdom og død': 1,
 'den dybe sorg': 1,
 'dybe sorg der': 1,
 'sorg der ramte': 1,
 'der ramte os': 1,
 'ramte os følte': 1,
 'os følte vi': 1,
 'følte vi at': 1,
 'vi at hele': 1,
 'at hele folket': 1,
 'hele folket tog': 1,
 'folket tog del': 1,
 'tog del i': 2,
 'del i og': 1,
 'i og jeg': 2,
 'og jeg kan': 4,
 'jeg kan ikke': 9,
 'kan ikke begynde': 2,
 'ikke begynde denne': 2,
 'begynde denne min': 1,
 'denne min første': 1,
 'min første nytårshilsen': 2,
 'første nytårshilsen uden': 1,
 'nytårshilsen uden at': 1,
 'uden at bringe': 2,
 'at bringe en': 2,
 'bringe en tak': 4,
 'en tak for': 33,
 'tak for

In [186]:
len(trigram_dict) # finder 4000 flere trigrams end sklearn???

100929

In [202]:
trigram_dict_forkortet = {}

for trigram in trigram_dict.keys():
    if trigram_dict[trigram] > 1:
        trigram_dict_forkortet[trigram] = trigram_dict[trigram] 

trigram_dict_forkortet

{'for min familie': 7,
 'min familie og': 16,
 'familie og for': 2,
 'og for mig': 2,
 'tog del i': 2,
 'i og jeg': 2,
 'og jeg kan': 4,
 'jeg kan ikke': 9,
 'kan ikke begynde': 2,
 'ikke begynde denne': 2,
 'min første nytårshilsen': 2,
 'uden at bringe': 2,
 'at bringe en': 2,
 'bringe en tak': 4,
 'en tak for': 33,
 'tak for al': 7,
 'for al den': 10,
 'al den varme': 2,
 'den varme og': 3,
 'varme og sympati': 2,
 'prins henrik og': 10,
 'henrik og mig': 3,
 'min moder dronning': 8,
 'moder dronning ingrid': 8,
 'som i de': 5,
 'i årets løb': 11,
 'og det har': 10,
 'det har været': 12,
 'en glæde og': 3,
 'glæde og inspiration': 2,
 'som danmarks dronning': 2,
 'på mange måder': 15,
 'blive husket for': 3,
 'husket for den': 2,
 'som det danske': 5,
 'det danske folk': 5,
 'danmarks medlemskab af': 2,
 'vi ind i': 2,
 'ind i et': 5,
 'samarbejde med de': 3,
 'vi går ind': 7,
 'går ind til': 3,
 'ind til det': 2,
 'til det med': 2,
 'om at dette': 2,
 'at dette vil': 2,
 'stille kr

In [204]:
len(trigram_dict_forkortet)

11560

# ALLE I ÉN LISTE

In [212]:
len(unigram_dict_forkortet) + len(bigram_dict_forkortet) + len(trigram_dict_forkortet)

32418

In [231]:
n_gram_dict = unigram_dict_forkortet | bigram_dict_forkortet | trigram_dict_forkortet

In [232]:
pprint(n_gram_dict)

{'000': 36,
 '000 er': 5,
 '000 er af': 4,
 '000 flere': 2,
 '000 kroner': 2,
 '000 mennesker': 3,
 '000 mio': 2,
 '000 mio kr': 2,
 '000 nye': 6,
 '000 nye jobs': 3,
 '000 unge': 2,
 '1': 11,
 '1 januar': 7,
 '1 januar er': 2,
 '10': 30,
 '10 000': 3,
 '10 000 er': 2,
 '10 20': 2,
 '10 af': 2,
 '10 af os': 2,
 '10 milliarder': 2,
 '10 millioner': 2,
 '10 år': 9,
 '10 år siden': 2,
 '10 års': 3,
 '100': 25,
 '100 000': 3,
 '100 000 er': 3,
 '100 procent': 2,
 '100 år': 11,
 '100 år siden': 7,
 '100 året': 4,
 '100 året for': 4,
 '1000': 3,
 '1000 år': 2,
 '11': 10,
 '11 september': 4,
 '11 september var': 2,
 '12': 12,
 '12 år': 2,
 '14': 6,
 '14 dage': 2,
 '14 år': 2,
 '15': 11,
 '15 februar': 2,
 '15 år': 6,
 '15 år tilbage': 2,
 '150': 4,
 '150 000': 2,
 '17': 3,
 '17 århundrede': 2,
 '18': 9,
 '18 år': 2,
 '18 års': 2,
 '18 års fødselsdag': 2,
 '1900': 2,
 '1920': 6,
 '1943': 3,
 '1945': 7,
 '1945 den': 2,
 '1950': 2,
 '1950 erne': 2,
 '1962': 2,
 '1970': 6,
 '1970 erne': 2,
 '1970

In [226]:
len(n_gram_dict)

32418

# Document-term matrix:

In [243]:
n_gram_keys_list = [key for key in n_gram_dict.keys()]

In [247]:
# sentences_list

['for min familie og for mig selv blev dette år skelsættende ved min fader kong frederiks sygdom og død',
 'den dybe sorg  der ramte os  følte vi  at hele folket tog del i  og jeg kan ikke begynde denne min første nytårshilsen uden at bringe en tak for al den varme og sympati  som blev prins henrik og mig  men ikke mindst min moder  dronning ingrid  til del',
 'de hjertelige følelser  som i de tunge dage støttede og løftede os  har jeg siden i årets løb mødt så mange gange  og det har været mig en glæde og inspiration i min gerning som danmarks dronning',
 '1972 vil blive et historisk år på mange måder',
 'i danmarkshistorien vil året blive husket for den betydningsfulde beslutning  som det danske folk traf om danmarks medlemskab af det udvidede europæiske fællesskab',
 'hermed træder vi ind i et nært samarbejde med de vigtigste vesteuropæiske nationer  vi går ind til det med store forhåbninger  men selvsagt også med en bevidsthed om  at dette vil stille krav til dansk kultur  og samfu

In [ ]:
document_term_matrix = np.empty((len(sentences_list),len(n_gram_keys_list)),dtype='uint8')

for i in range(len(sentences_list)):
    for j in range(len(n_gram_keys_list)):
        count = sentences_list[i].count(n_gram_keys_list[j])
        document_term_matrix[i,j] = count
        
document_term_matrix